# Global Malnutrition Analysis

In this project, we have analyzed several sources of data relevant to malnutrition amongst children ranging from the ages of infant to 5 years old. The major components of our analysis is as follows: 

1. Extracting data from various sources including UNICEF, World Bank and more (listed below) 
2. Combining malnutrition data into a single dataframe with information regarding wasting, overweight and stunting conditions amongst the global population of children from 1978 to 2021 
3. Analysing countries with the highest prevalence of malnutrition 
4. Developing a world heat map visualization to see the global malnutrition trend over the years 
5. Assessing a relationship, if any, between COVID deaths experienced by the country and prevalence of malnutrition among children

## Data Sources

1. The malnutrtion data is from UNICEF.org and it has the ratio of kids under 5 with stunt/severe stunting, wasted/severe wasted, and overweight problems at country-level from 1970 to 2021.(datalink: https://data.unicef.org/resources/data_explorer/unicef_f/?ag=UNICEF&df=GLOBAL_DATAFLOW&ver=1.0&dq=.NT_ANT_HAZ_NE2+NT_ANT_HAZ_NE3..&startPeriod=2016&endPeriod=2021). 

2. Country Geopandas data 

3. COVID deaths data 

4. Continent & Country Data 

## 1. Data Gathering

In [1]:
# download necessary packages

import numpy as np  #calculation
import pandas as pd  #dataframe
import scipy as sp #calculation
from scipy.interpolate import interp1d #smooth plot
import seaborn as sns #graph
import matplotlib.pyplot as plt #plot
from sklearn import linear_model #linear regression
import statsmodels.api as sm #linear regression model
from seaborn import heatmap #map
import os
import geopandas as gpd
import descartes
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'descartes'

In [ ]:
#download datasets and show headlines

In [ ]:
wasted_data = pd.read_csv("Wasted_Data.csv")
display(wasted_data[:3])

In [ ]:
# make sure the data downloaded is for wasting only
wasted_data['INDICATOR:Indicator'].unique() 

In [ ]:
ser_wasted_data=pd.read_csv("Severe_Wasted_Data.csv")
display(ser_wasted_data[:1])

In [ ]:
ser_wasted_data['INDICATOR:Indicator'].unique() #for severe wasting only

In [ ]:
stunted_data = pd.read_csv("Stunt_Data.csv")
display(stunted_data[:1])

In [ ]:
stunted_data['INDICATOR:Indicator'].unique() #stunting only

In [ ]:
ser_stunted_data = pd.read_csv("Severe_Stunt_Data.csv")
display(ser_stunted_data[:1])

In [ ]:
ser_stunted_data['INDICATOR:Indicator'].unique() #severe stunting only

In [ ]:
overweight_data = pd.read_csv("Overweight_Data.csv")
display(overweight_data[:1])

In [ ]:
overweight_data['INDICATOR:Indicator'].unique() #overweight only

## 2. Data Cleaning

In [ ]:
# data cleaning
# rename columns

# wasted
wasted_data = wasted_data.rename(columns = {"REF_AREA:Geographic area" : "Country", "SEX:Sex" : "Sex",\
                                         "TIME_PERIOD:Time period" : "Year", "OBS_VALUE:Observation Value" : "Waste_percent",\
                                           'AGE:Current age':'Age'})
# severe_wasted
ser_wasted_data=ser_wasted_data.rename(columns = {"REF_AREA:Geographic area" : "Country", "SEX:Sex" : "Sex",\
                                           "TIME_PERIOD:Time period" : "Year", "OBS_VALUE:Observation Value" : "Severe_Waste_percent",\
                                           'AGE:Current age':'Age'})
# stunted
stunted_data = stunted_data.rename(columns = {"REF_AREA:Geographic area" : "Country", "SEX:Sex" : "Sex",\
                                          "TIME_PERIOD:Time period" : "Year", "OBS_VALUE:Observation Value" : "Stunt_percent",\
                                            'AGE:Current age':'Age'})
# severe stunted
ser_stunted_data = ser_stunted_data.rename(columns = {"REF_AREA:Geographic area" : "Country", "SEX:Sex" : "Sex",\
                                          "TIME_PERIOD:Time period" : "Year", "OBS_VALUE:Observation Value" : "Severe_Stunt_percent",\
                                           'AGE:Current age':'Age'})
# overweight
overweight_data = overweight_data.rename(columns = {"REF_AREA:Geographic area" : "Country", "SEX:Sex" : "Sex",\
                                           "TIME_PERIOD:Time period" : "Year", "OBS_VALUE:Observation Value" : "Overweight_percent",\
                                            "INDICATOR:Indicator":"Overweight_Indicator",'AGE:Current age':'Age'})


In [ ]:
# Only keep the relevant data columns

wasted_df=wasted_data[['Country','Sex','Year','Waste_percent','Age']]
ser_wasted_df=ser_wasted_data[['Country','Sex','Year','Severe_Waste_percent','Age']]
stunted_df=stunted_data[['Country','Sex','Year','Stunt_percent','Age']]
ser_stunted_df=ser_stunted_data[['Country','Sex','Year','Severe_Stunt_percent','Age']]
overweight_df=overweight_data[['Country','Sex','Year','Overweight_percent','Age']]

In [ ]:
# combine with outer-join
combined_df=wasted_df.merge(ser_wasted_df,how='outer',on=['Country','Sex','Year','Age'])\
            .merge(stunted_df,how='outer',on=['Country','Sex','Year','Age'])\
            .merge(ser_stunted_df,how='outer',on=['Country','Sex','Year','Age'])\
            .merge(overweight_df,how='outer',on=['Country','Sex','Year','Age'])\
# shown sample lines
combined_df.head(3)

In [ ]:
# Calculate total waste percent column from adding percentage with normal waste condition and percentage with severe waste condition
combined_df['Waste_total_percent']=combined_df['Waste_percent']+combined_df['Severe_Waste_percent']
combined_df['Stunt_total_percent']=combined_df['Stunt_percent']+combined_df['Severe_Stunt_percent']
combined_df.head(1)

In [ ]:
# pop the previous waste & stunt columns, only keep the sum column
cols=['Waste_percent','Severe_Waste_percent','Stunt_percent','Severe_Stunt_percent']
for c in cols:
    combined_df.pop(c)
combined_df.head(1)

In [ ]:
# Observe properties of some columns
# 1. Is age all under 5?
combined_df['Age'].unique()
print('All data are for kids under 5 years old.')

In [ ]:
# Since the data only for kids under 5, we can delete this column 
del combined_df['Age']  

In [ ]:
# 2. Format sex column
combined_df['Sex'].unique()

In [ ]:
# only leave the "Total" for analysis
combined_df["Sex"] = combined_df["Sex"].str[1]
combined_df=combined_df[combined_df["Sex"]=='T']

In [ ]:
# 3. Format countries and regions
country_count=len(combined_df['Country'].unique())
print('There are a total number of '+ str(country_count) + ' countries/regions in the DataFrame initially.')

In [ ]:
# Format countries, separate to 3 digit ISO code and name
split_countries= combined_df["Country"].str.split(':', expand=True)

# join to combine df
combined_df = split_countries.join(combined_df)

# rename columns
combined_df = combined_df.rename(columns = {0 : "ISO Code", 1 : "Country Name"})

# format the region
combined_df["Country Name"] = combined_df["Country Name"].str.strip()

# filter only the countrys not regions
combined_df = combined_df[combined_df["ISO Code"].str.contains("_")==False]

# remove previous columns
del combined_df['Country']

combined_df.head(3)

In [ ]:
new_country_count=len(combined_df['Country Name'].unique())
print('There are now a total number of '+ str(new_country_count) + ' countries in the DataFrame now.')

In [ ]:
# 4. Observe the Year range
combined_df['Year'].unique()

In [ ]:
# format the year columns
# year remove some date, only keep year
combined_df["Year"] =  pd.to_datetime(combined_df["Year"]).dt.year
# observe new column values
years=combined_df['Year'].unique()
print(sorted(years))

In [ ]:
# 5. Drop rows with Na values
combined_df.dropna(inplace=True) #drop NaN

display(combined_df[:3])

In [ ]:
# create a new column of malnutrition score for mean of the five malnutrition factor
combined_df['Mean']=combined_df.iloc[:,4:9].mean(axis=1)
combined_df.head(3)

In [ ]:
# save the column names for future uses
mal_type=['Mean','Overweight_percent','Waste_total_percent','Stunt_total_percent']
#malnutrition type 0=mean
#malnutrition type 1=overweight
#malnutrition type 2=waste
#malnutrition type 3=stunt

## 3. Creating Visualizations

In [ ]:
# now observe overweight/waste/stunt 
fig, axis = plt.subplots(2,2) 
fig.suptitle("World's Children Malnutrition Histogram")

# Mean
axis[0,0].hist(combined_df[mal_type[0]],bins=30,color='deepskyblue')
axis[0,0].set_title("Mean")
# Overweight
axis[0,1].hist(combined_df[mal_type[1]],bins=30,color='deepskyblue')
axis[0,1].set_title("Overweight")
# Waste
axis[1,0].hist(combined_df[mal_type[2]],bins=30,color='deepskyblue')
axis[1,0].set_title("Waste")
# Stunt
axis[1,1].hist(combined_df[mal_type[3]],bins=30,color='deepskyblue')
axis[1,1].set_title("Stunt")

fig.set_size_inches(15, 10)
plt.show()

In [ ]:
# now observe overweight/waste/stunt 
fig, axis2 = plt.subplots(2,2) 
fig.suptitle("World's Children Malnutrition Boxplot")

# Mean
axis2[0,0].boxplot(combined_df[mal_type[0]])
axis2[0,0].set_title("Mean")

# Overweight
axis2[0,1].boxplot(combined_df[mal_type[1]])
axis2[0,1].set_title("Overweight")

# Waste
axis2[1,0].boxplot(combined_df[mal_type[2]])
axis2[1,0].set_title("Waste")

# Stunt
axis2[1,1].boxplot(combined_df[mal_type[3]])
axis2[1,1].set_title("Stunt")

fig.set_size_inches(15, 10)
plt.show()


In [ ]:
# observe correlation map
corr = combined_df.corr() # train is a pandas dataframe 
plt.figure(figsize = (8, 8))
heatmap(corr, cmap = 'Blues',annot=True)
plt.title('Correlation heatmap for all malnutrition types')
plt.show()

In [ ]:
# Now Observe US first
country_df=combined_df.copy()
country_df.index = country_df["ISO Code"] #make the iso code as index 

country_df.loc['USA'].plot(x='Year', y=['Overweight_percent','Waste_total_percent','Stunt_total_percent'])
plt.title('US Children Malnutrition')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
fig.set_size_inches(10, 10)
plt.show()


In [ ]:
# create a function to smooth the graph later
# use interpld from scipy to smooth the lines

def countryGraph(countryISO):
    countryData=country_df.loc[countryISO]
    country_name=countryData['Country Name'][1]
    x=countryData['Year']
    X_=np.linspace(x.min(), x.max(),300)
    for i in range(1,4):
        y=countryData[mal_type[i]]
        smooth_model = interp1d(x, y, kind = "cubic")
        f=smooth_model(X_)
        plt.plot(X_,f)
    plt.title(country_name+str(' Children Malnutrition Chart'))
    plt.legend(mal_type[1:])
    plt.show()
countryGraph('USA')

In [ ]:
# US has lower stunt and waste ratio but higher overweight problem for children under 5

In [ ]:
# groupby countries, find the country with highest mean malnutrition value
# because the minimum of cubic spline required datapoint is 4, we want to filter the country with 4 or more years of report
country_count=combined_df.groupby(by=['ISO Code','Country Name'],as_index=False).count()
country_count.rename(columns={'Mean':'Count'},inplace=True)
country_count=country_count[country_count['Count']>=4]
country_count=country_count[['ISO Code','Count']]
display(country_count[:3])

In [ ]:
#country_rank.sort_values(by='Mean',ascending=True,inplace=True)
country_rank=combined_df.groupby(by=['ISO Code','Country Name'],as_index=False).mean()
country_rank=country_rank.merge(country_count,how='inner',on='ISO Code')
country_rank=country_rank.sort_values(by='Mean')
display(country_rank[:3])

In [ ]:
# keep a record of the top 5 and bot 5 country with S&W mean for observation
top5=country_rank[:5]['Country Name'].tolist()
display(top5)
# the five country with least percentages of Stunt & Waste mean


In [ ]:
top5ISO=country_rank[:5]['ISO Code'].tolist()
display(top5ISO)
# 5 countries' corresponding iso code


In [ ]:
# Plot graphs to observe the trend for each malnutrition problem at country level for top 5 countries
#for i in top5ISO:
#    countryGraph(i)
    
#plt.show()
### ValueError: Expect x to not have duplicates??

In [ ]:
# Plot graphs to observe the trend for each malnutrition problem at country level for top 5 countries

for c in zip(top5,top5ISO):
    country_df.loc[c[1]].plot(x='Year', y=mal_type[1:])
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.title(c[0])
plt.show()


In [ ]:
bot5=country_rank[-5:]['Country Name'].tolist()
display(bot5) # the five country with high percentages of Stunt & Waste mean

In [ ]:
bot5ISO=country_rank[-5:]['ISO Code'].tolist()
display(bot5ISO)

In [ ]:
#graph bottom 5 countries malnutrition graphs
for i in bot5ISO:
    countryGraph(i)


### Continent Analysis

In [ ]:
continent = pd.read_csv("countryContinent.csv")

continent = continent[["country", "continent"]]

joined = country_rank.merge(continent, left_on = "Country Name", right_on = "country")

joined = joined.drop(columns = "country")

In [ ]:
display(joined[:3])

In [ ]:
by_continent = joined.groupby(["Year", "continent"], as_index = False).mean()

display(by_continent[:3])

In [ ]:
sns.lineplot(x="Year", y="Mean",
             hue="continent",data=by_continent, palette="husl", lw = 3).set(title = "% of Under 5 Year-Old with Wasting")

sns.set(rc={'figure.figsize':(15,6)})

sns.set_style("white")

In [ ]:
## SHEENA TRYING SMOOTH THIS
#xc=by_continent['Year']
#Xc_=np.linspace(xc.min(), xc.max(),300)

#yc=by_continent['Mean']
#f1= interp1d(xc, yc, kind = "cubic")
#yc_=f1(Xc)

# Plotting the Graph
#plt.plot(Xc_, yc_)
# not working becuase x was repeated?? 
# https://stackoverflow.com/questions/68100959/plot-smooth-curve-with-duplicate-values-in-list

## Mapping

In [ ]:
# import map
world_map = gpd.read_file(os.getcwd()+'\\World_Map\\World_Countries__Generalized_.shp')

In [ ]:
map_all = world_map.merge(combined_df, how='left', left_on='COUNTRYAFF', right_on='Country Name')
print('Verifying data has combined')
display(map_all.head(1))

In [ ]:
print('Overweight observations range', np.nanmin(map_all['Overweight_percent'].values), '% to', np.nanmax(map_all['Overweight_percent'].values), '%.')
print('Wasted observations range', np.nanmin(map_all['Waste_total_percent'].values), '% to', np.nanmax(map_all['Waste_total_percent'].values), '%.')
print('Stunted observations range', np.nanmin(map_all['Stunt_total_percent'].values), '% to', np.nanmax(map_all['Stunt_total_percent'].values), '%.')

In [ ]:
# map with all overweight indicators
vmin, vmax = 0, np.nanmax(map_all['Overweight_percent'].values)

figo, axo = plt.subplots(1, figsize = (16,12))
map_all.plot(column='Overweight_percent', cmap='OrRd', ax=axo, linewidth=0.2, edgecolors='0.3')
bar_infoo = plt.cm.ScalarMappable(cmap='OrRd', norm=plt.Normalize(vmin=vmin, vmax=vmax))
axo.set_title('Percent of overweight children under age 5 around the world', fontdict={'fontsize':20})
axo.set_axis_off()
bar_infoo._A = []
cbar = figo.colorbar(bar_infoo, orientation='horizontal')

In [ ]:
# map with all waste indicators
vmin, vmax = 0, np.nanmax(map_all['Waste_total_percent'].values)

figw, axw = plt.subplots(1, figsize = (16,12))
map_all.plot(column='Waste_total_percent', cmap='YlGnBu', ax=axw, linewidth=0.2, edgecolors='0.3')
bar_infow = plt.cm.ScalarMappable(cmap='YlGnBu', norm=plt.Normalize(vmin=vmin, vmax=vmax))
axw.set_title('Percent of wasted children under age 5 around the world', fontdict={'fontsize':20})
axw.set_axis_off()
bar_infow._A = []
cbar = figw.colorbar(bar_infow, orientation='horizontal')

In [ ]:
# map with all stunt indicators
vmin, vmax = 0, np.nanmax(map_all['Stunt_total_percent'].values)

figs, axs = plt.subplots(1, figsize = (16,12))
map_all.plot(column='Stunt_total_percent', cmap='RdPu', ax=axs, linewidth=0.2, edgecolors='0.3')
bar_infos = plt.cm.ScalarMappable(cmap='RdPu', norm=plt.Normalize(vmin=vmin, vmax=vmax))
axs.set_title('Percent of stunted children under age 5 around the world', fontdict={'fontsize':20})
axs.set_axis_off()
bar_infos._A = []
cbar = figs.colorbar(bar_infos, orientation='horizontal')

## Animation

In [ ]:
# list of years
min_year = int(map_all['Year'].min())
max_year = int(map_all['Year'].max())+1
list_of_years = list(range(min_year, max_year))
print('Years range from', min(list_of_years), 'to', max(list_of_years))

In [ ]:
# list of countries
countries = map_all['COUNTRYAFF'].unique()
print('There are', len(countries), 'countries listed.')

In [ ]:
# save all the maps in the pictures folder
output_path = os.getcwd()+'\\pictures'
filenames = []

In [ ]:
# algorithm to make a map for each year and save as jpg
def make_maps(map_col, color, ax, list_of_years = list(range(2000,2020))):
    # start the for loop to create one map per year
    for year in list_of_years:
        # narrow the data to the year desired
        yearly = combined_df.loc[combined_df['Year'] == year].reset_index(drop=True)
        maps = world_map.merge(yearly, how='left', left_on='COUNTRYAFF', right_on='Country Name')
        
        # normalize across all years
        vmin = min(combined_df[map_col])
        vmax = max(combined_df[map_col])

        # create map
        fig = maps.plot(column=map_col,
                        cmap=color,
                        figsize=(16,12),
                        linewidth=0.3,
                        edgecolor='0.3',
                        ax=ax,
                        norm=plt.Normalize(vmin=vmin, vmax=vmax))
        
        # add a title
        fig.set_title(map_col + ' children under age 5 in ' + str(year), \
                  fontdict={'fontsize': '20',
                             'fontweight' : '3'})

        # remove axis of chart
        fig.set_axis_off()
        
        # this will save the figure as a high-res jpg
        filename = str(year)+'_'+map_col+'.jpg'
        filepath = os.path.join(output_path, filename)
        filenames.append(filename)
        chart = fig.get_figure()
        chart.savefig(filepath, dpi=300)

In [ ]:
make_maps('Overweight_percent', 'OrRd', axo, list_of_years)

In [ ]:
make_maps('Waste_total_percent', 'YlGnBu', axw, list_of_years)

In [ ]:
make_maps('Stunt_total_percent', 'RdPu', axs, list_of_years)

In [ ]:
# pull the created pictures into a gif
import glob
from PIL import Image

def make_gif(fp_in, fp_out, duration = 400, loop = 3):
    img, *imgs = [Image.open(f) for f in sorted(glob.glob(fp_in))]
    img.save(fp=fp_out, format='GIF', append_images=imgs,
             save_all=True, duration=duration, loop=loop)

In [ ]:
# filepaths for overweight
o_fp_in = os.getcwd()+'\\pictures\\*_Overweight*.jpg'
o_fp_out = os.getcwd()+'\\overweight.gif'

#make overweight gif
make_gif(o_fp_in, o_fp_out)

In [ ]:
# filepaths for wasted
w_fp_in = os.getcwd()+'\\pictures\\*_Waste*.jpg'
w_fp_out = os.getcwd()+'\\waste.gif'

#make wasted gif
make_gif(w_fp_in, w_fp_out)

In [ ]:
# filepaths for stunted
s_fp_in = os.getcwd()+'\\pictures\\*_Stunt*.jpg'
s_fp_out = os.getcwd()+'\\stunt.gif'

#make stunted gif
make_gif(s_fp_in, s_fp_out)

![SegmentLocal](overweight.gif "segment")

![SegmentLocal](waste.gif "segment")

![SegmentLocal](stunt.gif "segment")

### Covid Analysis

In [ ]:
# combine with COVID

In [ ]:
###### 
# Download covid vaccine rate data to covid_data from https://ourworldindata.org/covid-vaccinations
covid_data=pd.read_csv("owid-covid-data.csv")
covid_data.tail(5)

In [ ]:
#before 12/31/2020, the total number of cases for each countries
covid_data=covid_data[covid_data['date']<'2021-01-01']
covid_2020=covid_data.groupby(by='iso_code').max()
display(covid_2020[:5])


In [ ]:
norm_covid_2020=covid_2020.copy()
# since cases number varies alot, apply z-score standardlization to total_cases column to normalize the case number
norm_covid_2020=norm_covid_2020.sort_values(by='total_cases')
norm_covid_2020['norm_total_cases'] = (norm_covid_2020['total_cases'] - norm_covid_2020['total_cases'].mean())\
                                    / (norm_covid_2020['total_cases'].std())    

display(norm_covid_2020)



In [ ]:
#Observed the world data on last row, remove for outlier data point
norm_covid_2020=norm_covid_2020[:-1] 
display(norm_covid_2020)
plt.hist(norm_covid_2020['norm_total_cases'],bins=50)
plt.show()


#### Highest % of COVID deaths in the world

In [ ]:
covid = pd.read_csv("owid-covid-data.csv")[["location", "date", "new_deaths"]]

covid["date"] = pd.to_datetime(covid["date"]).dt.year

top_10_covid = covid.groupby(["location", "date"], as_index = False).sum()

display(top_10_covid[:3])

In [ ]:
worldpop = pd.read_csv("Worldpop.csv")

worldpop["PopTotal"] = worldpop["PopTotal"].astype(int)

display(worldpop[:3])

Join total population to covid deaths for comparison

In [ ]:
top_10_covid = worldpop.merge(top_10_covid, left_on = ["Location", "Time"], right_on = ["location", "date"])

top_10_covid = top_10_covid.drop(columns = ["date", "location"])

In [ ]:
display(top_10_covid[:3])

In [ ]:
# population total in 1000's 

top_10_covid["deaths percentage"] = (top_10_covid["new_deaths"]/(top_10_covid["PopTotal"]*1000))

top_10_covid = top_10_covid[top_10_covid["Location"] != "World"]

covid_countries = top_10_covid.sort_values(by = "deaths percentage", ascending = False)

In [ ]:
covid_countries = covid_countries[:10]

display(covid_countries[:3])

In [ ]:
covid_trend = top_10_covid[top_10_covid["Location"].isin(covid_countries["Location"].unique())]

display(covid_trend[:3])

In [ ]:
sns.lineplot(x="Time", y="deaths percentage",
             hue="Location",data=covid_trend, palette="Set2", lw =3).set(title = "Countries with highest Covid-19 % of Deaths")

sns.set(rc={'figure.figsize':(15,6)})

sns.set_style("white")

In [ ]:
# countries with most covid deaths 
continent = pd.read_csv("countryContinent.csv")

continent = continent[["country", "continent"]]

covid_deaths = covid_trend.merge(continent, left_on = "Location", right_on = "country")

covid_deaths = covid_deaths.drop(columns = ["country"])

display(covid_deaths["continent"].unique())